# Progetto Computer Science

*Nome:* Christian

*Cognome:* Uccheddu

*Matricola:* 800428


# Setup dell'ambiente

Importazione di tutte le librerie utili al funzionamento del programma e settaggio dei path in cui sono presenti i dati in modo da essere facilmente cambiabili all'inizio del programma.

In [1]:
import pandas as pd
import datetime
import numpy as np

In [ ]:
#Dataset: "loans.csv"
loans_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans.csv"
#Dataset: "lenders.csv"
lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/lenders.csv"
#Dataset: "loans_lenders.csv"
loans_lenders_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/loans_lenders.csv"
#Dataset: "country_stats.csv"
country_stats_path = r"/home/christian/Scrivania/progetti/cs_project/dataset/country_stats.csv" 

**1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.**